In [ ]:
pip install requests beautifulsoup4

In [ ]:
!pip install selenium
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb && apt install ./google-chrome-stable_current_amd64.deb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
--2023-10-02 19:44:59--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 74.125.204.93, 74.125.204.91, 74.125.204.136, ...
Connecting to dl.google.com (dl.google.com)|74.125.204.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104035880 (99M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>]  99.22M   110MB/s    in 0.9s    

2023-10-02 19:45:00 (110 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [104035880/104035880]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'google-chrome-stable' instead of './go

In [ ]:
def countNumberOfSchools(county, district):
  url = f'https://www.ed-data.org/district/{county}/{district}'
  districtDriver = webdriver.Chrome(options=options)
  districtDriver.get(url)
  button = districtDriver.find_element(By.ID, "entity-list-btn")
  button.click()
  table = districtDriver.find_element(By.ID, "DataTables_Table_0")
  tableBody = table.find_element(By.TAG_NAME, "tbody")
  schools = tableBody.find_elements(By.TAG_NAME, "tr")
  return len(schools)

In [3]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.select import Select

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

#https://blog.devgenius.io/use-selenium-webdriver-in-google-colab-d5f2dba1d9f5


# Variables for the website URL
county = 'Del-Norte'

countyUrl = f'https://www.ed-data.org/county/{county}'
districts = []
countyResponse = requests.get(countyUrl)


# Check if the request was successful
if countyResponse.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(countyResponse.content, 'html.parser')

    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    #overcome limited resource problems
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")

    # sheet = pd.read_excel('/content/CA All School Districts Spreadsheet - Need and Equity Data.xlsx', index_col=0)
    # pollutionInfoTable = sheet[["County", "District", "Average Pollution Burden ", "Average Pollution Burden Percentile"]].rename(columns={"Average Pollution Burden ":"Average Pollution Burden"})

    #get all counties
    counties = []
    option_elements = [option for option in soup.find_all('option') if option.parent.name == 'select']
    for option in option_elements[1:]:
      county = option.text.strip()
      counties.append(county)

    #for every county, find all the districts and their information
    for county in counties:
      countyUrl = f'https://www.ed-data.org/county/{county}'
      driver = webdriver.Chrome(options=options)
      driver.get(countyUrl)
      button = driver.find_element(By.ID, "entity-list-btn")
      button.click()
      table = driver.find_element(By.ID, "DataTables_Table_0")
      tableBody = table.find_element(By.TAG_NAME, "tbody")
      elementList = tableBody.find_elements(By.TAG_NAME, "tr")
      for element in elementList:
        district = element.find_elements(By.TAG_NAME, "td")[0]
        district = district.text

        url = f'https://www.ed-data.org/district/{county}/{district}'

        # Send a GET request to the website
        response = requests.get(url)

        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the div element with class "profiledata-container key-att"
        div_element = soup.find('div', class_='profiledata-container key-att')

        # Find the ul elements within the div_element
        if not div_element or "SBE" in district:
          print("District: ", county, ', ', district)
          print("---------------------")
          print("Could not find data, please manually check")
          print("\n")
          continue
        ul_elements = div_element.find_all('ul', class_='profiledata')

        # Define a dictionary to store the information
        district_info = {}

        # Loop through the ul elements to collect the information
        for ul in ul_elements:
            li_elements = ul.find_all('li')
            if len(li_elements) == 2:
                key = li_elements[0].text.strip()
                value = li_elements[1].find('span').text.strip()
                district_info[key] = value
        if district_info["District Type"] == "County Office of Education (COE)" or "charter" in district.lower():
          print("District: ", county, ', ', district)
          print("---------------------")
          for key, value in district_info.items():
            if key != 'CDS Code':
              print(f"{key}: {value}")
          print("\n")
          continue


        #Get number of schools in a district
        districtDriver = webdriver.Chrome(options=options)
        districtDriver.get(url)
        button = districtDriver.find_element(By.ID, "entity-list-btn")
        button.click()
        table = districtDriver.find_element(By.ID, "DataTables_Table_0")
        tableBody = table.find_element(By.TAG_NAME, "tbody")
        schoolElements = tableBody.find_elements(By.TAG_NAME, "tr")
        district_info["Number of Schools (2021-2022)"] = len(schoolElements)

        #Get Total # of Jurisdictions Per School District and Jurisdiction Name
        if len(schoolElements) == 0:
          district_info["Total # of Jurisdictions Per School District"] = "Error, please manually check"
          district_info["Jurisdiction Name"] = []
        elif schoolElements[0].find_elements(By.TAG_NAME, "td")[0].text == "": #No matching records found
          district_info["Total # of Jurisdictions Per School District"] = "0 - No schools found"
          district_info["Jurisdiction Name"] = []
        else:
          schools = [school.find_elements(By.TAG_NAME, "td")[2] for school in schoolElements]
          jurisdictions = [jurisdiction.text for jurisdiction in schools]
          uniqueJurisdictions = np.unique(jurisdictions)
          district_info["Total # of Jurisdictions Per School District"] = len(uniqueJurisdictions)
          district_info["Jurisdiction Name"] = uniqueJurisdictions

        #Get %unduplicated
        try:
          chart = districtDriver.find_element(By.ID, "chartContent-22")
          div0 = chart.find_elements(By.TAG_NAME, "div")[0]
          div1 = div0.find_element(By.ID, "charttab") #charttab
          div2 = div1.find_elements(By.TAG_NAME, "div")[0] #chartdatamenu
          div3 = div2.find_elements(By.TAG_NAME, "div")[0] #table-div
          table = div3.find_elements(By.TAG_NAME, "table")[0]
          tbody = table.find_elements(By.TAG_NAME, "tbody")[0]
          tr = tbody.find_elements(By.TAG_NAME, "tr")[0]
          td = tr.find_elements(By.TAG_NAME, "td")[5]
          span = td.find_elements(By.TAG_NAME, "span")[1]
          district_info["% Unduplicated"] = span.get_attribute("innerHTML")
        except Exception as e:
          district_info["% Unduplicated"] = "Demographic data not available, please manually check"

        #Get % FRM
        try:
          chart = districtDriver.find_element(By.ID, "chartContent-23")
          div0 = chart.find_elements(By.TAG_NAME, "div")[0]
          div1 = div0.find_element(By.ID, "charttab") #charttab
          div2 = div1.find_elements(By.TAG_NAME, "div")[0] #chartdatamenu
          div3 = div2.find_elements(By.TAG_NAME, "div")[0] #table-div
          table = div3.find_elements(By.TAG_NAME, "table")[0]
          tbody = table.find_elements(By.TAG_NAME, "tbody")[0]
          tr = tbody.find_elements(By.TAG_NAME, "tr")[0]
          td = tr.find_elements(By.TAG_NAME, "td")[5]
          span = td.find_elements(By.TAG_NAME, "span")[1]
          district_info["% FRM"] = span.get_attribute("innerHTML")
        except Exception as e:
          district_info["% FRM"] = "Demographic data not available, please manually check"

        #Get % English Leaners
        try:
          chart = districtDriver.find_element(By.ID, "chartContent-24")
          div0 = chart.find_elements(By.TAG_NAME, "div")[0]
          div1 = div0.find_element(By.ID, "charttab") #charttab
          div2 = div1.find_elements(By.TAG_NAME, "div")[0] #chartdatamenu
          div3 = div2.find_elements(By.TAG_NAME, "div")[0] #table-div
          table = div3.find_elements(By.TAG_NAME, "table")[0]
          tbody = table.find_elements(By.TAG_NAME, "tbody")[0]
          tr = tbody.find_elements(By.TAG_NAME, "tr")[0]
          td = tr.find_elements(By.TAG_NAME, "td")[5]
          span = td.find_elements(By.TAG_NAME, "span")[1]
          district_info["% English Leaners"] = span.get_attribute("innerHTML")
        except Exception as e:
          district_info["% English Leaners"] = "Demographic data not available, please manually check"

        #Get Average Pollution Burden and Average Pollution Burden Percentile
        avgPollutionBurden = pollutionInfoTable[pollutionInfoTable["District"] == district]["Average Pollution Burden"]
        district_info["Average Pollution Burden"] = avgPollutionBurden[0] if len(avgPollutionBurden) > 0 else "N/A"
        avgPollutionBurdenPercentile = pollutionInfoTable[pollutionInfoTable["District"] == district]["Average Pollution Burden Percentile"]
        district_info["Average Pollution Burden Percentile"] = avgPollutionBurdenPercentile[0] if len(avgPollutionBurdenPercentile) > 0 else "N/A"


        # Print the collected information
        print("District: ", county, ', ', district)
        print("---------------------")
        for key, value in district_info.items():
          if key != 'CDS Code':
            print(f"{key}: {value}")
        print("\n")
    else:
        print('Failed to retrieve data. Status code:', response.status_code)

    # option_elements = [option for option in soup.find_all('option') if option.parent.name == 'select']

District:  Alameda ,  Alameda County Office of Education
---------------------
District Type: County Office of Education (COE)
Grade Span: K-12
2022-23 Census Day Enrollment: 3,755




NameError: name 'pollutionInfoTable' is not defined

In [ ]:
#Get # of Certified Teachers (2018-2019)
        #---
        # navigationBar = districtDriver.find_element(By.ID, "horizontalTab")
        # navigationBarUl = navigationBar.find_elements(By.TAG_NAME, "ul")[0]
        # li0 = navigationBarUl.find_elements(By.TAG_NAME, "li")[0]
        # li = navigationBarUl.find_elements(By.TAG_NAME, "li")[1]
        #---
        # staffButton.click()
        # viewTableDateButton = districtDriver.find_element(By.ID, "datatable-165")
        # viewTableDateButton.click()
        # table = districtDriver.find_elements(By.CLASS_NAME, "table footable table-bordered master-footable")[0]
        # tableBody = table.find_elements(By.TAG_NAME, "tbody")
        # table = districtDriver.find_elements(By.CLASS_NAME, "chart-data-row total-line")[0]
        # numberWrapper = table.find_elements(By.TAG_NAME, "td")[2]
        # number = numberWrapper.find_elements(By.TAG_NAME, "span")[0].text
        # district_info["# of Certificated Teachers (2018-19)"] = number
        #---
        # districtDriver.execute_script("arguments[0].setAttribute('class', 'tab-label resp-tab-item hor_1')", li0)
        # districtDriver.execute_script("arguments[0].setAttribute('class', 'tab-label resp-tab-item hor_1 resp-tab-active')", li)
        # div = districtDriver.find_element(By.ID, "packeryContainer-11")
        # districtDriver.execute_script("arguments[0].setAttribute('style.display', 'block')", div)
        #---

        #---
        # staffTab = districtDriver.find_element(By.ID, "staff-data-tab")
        # div0 = staffTab.find_elements(By.TAG_NAME, "div")[0] #accordion-content
        # section = div0.find_elements(By.TAG_NAME, "section")[0]
        #---

        # div = section.find_elements(By.TAG_NAME, "div")[0]

        #---
        # div1 = section.find_elements(By.TAG_NAME, "div")[0] #packeryContainer-11
        # print(div1.get_attribute("id"))
        # print(div1.find_elements(By.XPATH, ".//*"))
        # div = div1.find_elements(By.TAG_NAME, "div") #packeryContainer-11
        # print(div)
        # div2 = div.find_elements(By.TAG_NAME, "div")[0] #chart-image
        # div3 = div2.find_elements(By.TAG_NAME, "div")[1] #chartContent
        # div4 = div3.find_elements(By.TAG_NAME, "div")[0] #
        #---

        # chart = districtDriver.find_element(By.ID, "chartContent-165")
        # div0 = chart.find_elements(By.TAG_NAME, "div")[0]

        #---
        # div1 = div4.find_element(By.ID, "charttab") #charttab
        # div2 = div1.find_elements(By.TAG_NAME, "div")[0] #chartdatamenu
        # div3 = div2.find_elements(By.TAG_NAME, "div")[0] #table-div
        # table = div3.find_elements(By.TAG_NAME, "table")[0]
        # tbody = table.find_elements(By.TAG_NAME, "tbody")[0]
        # tr = tbody.find_elements(By.TAG_NAME, "tr")[0]
        # td = tr.find_elements(By.TAG_NAME, "td")[2]
        # span = td.find_elements(By.TAG_NAME, "span")[1]
        # district_info["# of Certificated Teachers (2018-19)"] = span.get_attribute("innerHTML")
        #---

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver



# Variables for the website URL
county = 'Alameda'

countyUrl = f'https://www.ed-data.org/county/{county}'
districts = []
countyResponse = requests.get(countyUrl)

# Check if the request was successful
if countyResponse.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(countyResponse.content, 'html.parser')

    # Find the div element with class "profiledata-container key-att"
    #div_element = soup.find('select', class_='chosen-select-deselect cds').findAll("option")
    # div_element = soup.find('select', {"id":"districtddl"}).find_all("option")
    # div_element = soup.find('div', class_="col-md-4", id="cdsdistrict")

    # div_element = soup.find('div', class_="col-md-4", id="cdsdistrict")
    # inner_element = div_element.find_all("select")[0]
    # # print(div_element)
    # # print(inner_element.find_all("option"))
    # option_elements = [select for select in soup.find_all("td")]
    # # option_elements = inner_element.find_all("option")
    # print(option_elements)
    # for option in option_elements[1:]:
    #   district = option.text.strip()
    #   districts.append(district)

    # div_element = soup.find('select', class_='chosen-select-deselect cds')
    #this gets the districts
    option_elements = [option for option in soup.find_all('option') if option.parent.name == 'select']
    for option in option_elements[1:]:
      district = option.text.strip()
      districts.append(district)

    # Find the ul elements within the div_element
    # div_element = soup.find('div', class_='chosen-drop')
    # district_elements = div_element.find('ul', class_="chosen-results")
    # for li in district_elements:
    #   district = li.text.strip()
    #   districts.append(district)

    # Define a dictionary to store the information
print(districts)

district = 'Hayward-Unified'

# URL of the Ed-Data website for the specified district
url = f'https://www.ed-data.org/district/{county}/{district}'

# Send a GET request to the website
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the div element with class "profiledata-container key-att"
    div_element = soup.find('div', class_='profiledata-container key-att')

    # Find the ul elements within the div_element
    ul_elements = div_element.find_all('ul', class_='profiledata')

    # Define a dictionary to store the information
    district_info = {}

    # Loop through the ul elements to collect the information
    for ul in ul_elements:
        li_elements = ul.find_all('li')
        if len(li_elements) == 2:
            key = li_elements[0].text.strip()
            value = li_elements[1].find('span').text.strip()
            district_info[key] = value

    # Print the collected information
    print("District: ", county, ', ', district)
    print("---------------------")
    for key, value in district_info.items():
      if key != 'CDS Code':
        print(f"{key}: {value}")
else:
    print('Failed to retrieve data. Status code:', response.status_code)


[<option value="0">Select a County</option>, <option value="1">Alameda</option>, <option value="2">Alpine</option>, <option value="3">Amador</option>, <option value="4">Butte</option>, <option value="5">Calaveras</option>, <option value="6">Colusa</option>, <option value="7">Contra Costa</option>, <option value="8">Del Norte</option>, <option value="9">El Dorado</option>, <option value="10">Fresno</option>, <option value="11">Glenn</option>, <option value="12">Humboldt</option>, <option value="13">Imperial</option>, <option value="14">Inyo</option>, <option value="15">Kern</option>, <option value="16">Kings</option>, <option value="17">Lake</option>, <option value="18">Lassen</option>, <option value="19">Los Angeles</option>, <option value="20">Madera</option>, <option value="21">Marin</option>, <option value="22">Mariposa</option>, <option value="23">Mendocino</option>, <option value="24">Merced</option>, <option value="25">Modoc</option>, <option value="26">Mono</option>, <option val

## Getting Demographic Data from district

read the couple comments below to get the data first

In [ ]:
# Go to the LEA Level CALPADS UPC data in the google sheet -> File ->download as csv
# run this to select the downloaded CALPADS UPC data csv
from google.colab import files


uploaded = files.upload()

KeyboardInterrupt: ignored

In [ ]:
#replace the name of the file with whatever it is named on your computer... then just run the rest to get the values!!
import pandas as pd
import io

UPC_df = pd.read_csv(io.BytesIO(uploaded['calpadsUPC20212022 (4).csv']))

In [ ]:
UPC_df.replace(',','', regex=True, inplace=True)
data_types_dict = {'Free &\nReduced\nMeal\nProgram': int,
                   'Total \nEnrollment': int,
                   'CALPADS Unduplicated \nPupil Count\n(UPC)': int,
                   'English \nLearner \n(EL)': int}


UPC_df = UPC_df.astype(data_types_dict)
UPC_df = UPC_df.loc[:, ['County Name', 'District Name', 'Free &\nReduced\nMeal\nProgram', 'Total \nEnrollment', 'CALPADS Unduplicated \nPupil Count\n(UPC)', 'English \nLearner \n(EL)']]

In [ ]:
def get_district_demographic_info(county, district):
  grouped_by_district = UPC_df.groupby(['County Name', 'District Name']).sum().reset_index()
  our_district = grouped_by_district[(grouped_by_district['County Name'] == county) & (grouped_by_district['District Name'] == district)]
  percent_FRM = float((our_district['Free &\nReduced\nMeal\nProgram'] / our_district['Total \nEnrollment']) *100)
  percent_UPC = float((our_district['CALPADS Unduplicated \nPupil Count\n(UPC)'] / our_district['Total \nEnrollment']) *100)
  percent_EngLearners = float((our_district['English \nLearner \n(EL)'] / our_district['Total \nEnrollment']) *100)
  print("%Unduplicated : " + str(percent_UPC) + "\n %FRM : " +str(percent_FRM) + "\n %English Learners : " + str(percent_EngLearners))

In [ ]:
get_district_demographic_info('Sonoma', 'Bellevue Union')

In [ ]:
# Variables for the website URL

county = 'Sonoma'
district = 'Bellevue Union'

# URL of the Ed-Data website for the specified district
url = f'https://www.ed-data.org/district/{county}/{district}'

# Send a GET request to the website
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    headers = {"X-Requested-With": "XMLHttpRequest", 'Accept': 'text/html'}

    # URL to the html table with the General Revenue data. This URL is dynamically requested by the District website
    api_url = "http://www.ed-data.org/Chart/Generate/49-70615-0000000/2018/2022/157?drillinfilter="

    # GET the district website, and GET the dynamically embedded HTML General Revenue table from that
    with requests.Session() as s:
      soup = BeautifulSoup(s.get(url).content, "html.parser")
      soup2 = BeautifulSoup(s.get(api_url, headers=headers).text, "html.parser")
      df = pd.read_html(s.get(api_url, headers=headers).text)
      #print(df)
      df = pd.DataFrame(df[0])
      revenue = df['2020-21'][2]

    print("District: ", county, ', ', district)
    print("---------------------")
    print(f"Expense of Education per ADA 2020-2021: {revenue}")


else:
   print('Failed to retrieve data. Status code:', response.status_code)

# Difficult because of the fact that this value is dynamically loaded by the website, meaning it appears when clicked on.
# Basically,  The table is not always a part of the website like the profile data is , which makes it slightly more annoying,
# but i feel like im getting close

District:  Sonoma ,  Bellevue Union
---------------------
Expense of Education per ADA 2020-2021: 16286


In [ ]:
pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.3/399.3 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
import time

county = 'Sonoma'
district = 'Bellevue Union'

# URL of the Ed-Data website for the specified district
url = f'https://www.ed-data.org/district/{county}/{district}'


service = ChromeService(executable_path='C:/Users/nathan/Downloads/chromedriver_mac64/chromedriver')


driver = webdriver.Chrome(service=service)
# initiating the webdriver. Parameter includes the path of the webdriver.
#driver = webdriver.Chrome('C:/Users/nathan/Downloads/chromedriver_mac64/chromedriver')
driver.get(url)

# this is just to ensure that the page is loaded
time.sleep(5)

html = driver.page_source

# this renders the JS code and stores all
# of the information in static HTML code.

# Now, we could simply apply bs4 to html variable
soup = BeautifulSoup(html, "html.parser")




WebDriverException: ignored

In [ ]:
pip install webdriver-manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

# instantiate options
options = webdriver.ChromeOptions()

# run browser in headless mode
options.headless = True

options.binary_location = "C:/Macintosh HD/Applications/Google Chrome/chrome.exe"
# instantiate driver
driver = webdriver.Chrome(service=ChromeService(
	ChromeDriverManager().install()), options=options)

county = 'Sonoma'
district = 'Bellevue Union'

# URL of the Ed-Data website for the specified district
url = f'https://www.ed-data.org/district/{county}/{district}'


# get the entire website content
driver.get(url)

time.sleep(5)

html = driver.page_source

# this renders the JS code and stores all
# of the information in static HTML code.

# Now, we could simply apply bs4 to html variable
soup = BeautifulSoup(html, "html.parser")

<ipython-input-16-e1b05e6bddbf>:10: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


WebDriverException: ignored